# Pre-processing pipeline for spikeglx sessions, zebra finch/starling
- For every run in the session:
 - Load the recordings
 - Extract wav chan with micrhopohone and make a wav chan with the nidq syn signal
 - Get the sync events for the nidq sync channel
 
 - Do bout detection
 
In another notebook, bout detection is curated
- Left to decide where to:
    - Sort spikes
    - Sync the spikes/lfp/nidq
    - make and plot 'bout rasters'

In [1]:
%matplotlib inline
import os
import glob
import logging
import pickle
import numpy as np
import pandas as pd
from scipy.io import wavfile
from scipy import signal
import traceback
import warnings
import socket

from matplotlib import pyplot as plt
from importlib import reload

logger = logging.getLogger()
handler = logging.StreamHandler()
formatter = logging.Formatter(
        '%(asctime)s %(name)-12s %(levelname)-8s %(message)s')
handler.setFormatter(formatter)
logger.addHandler(handler)
logger.setLevel(logging.INFO)
logger.info('Running on {}'.format(socket.gethostname()))

2023-03-07 11:28:37,117 root         INFO     Running on pakhi.ucsd.edu


In [2]:
from ceciestunepipe.file import bcistructure as et
from ceciestunepipe.util import sglxutil as sglu
from ceciestunepipe.util import rigutil as ru
from ceciestunepipe.util import wavutil as wu
from ceciestunepipe.util import syncutil as su

from ceciestunepipe.util.sound import boutsearch as bs
from ceciestunepipe.pipeline import searchbout as sb

from ceciestunepipe.util.spikeextractors import preprocess as pre
from ceciestunepipe.util.spikeextractors.extractors.spikeglxrecordingextractor import readSGLX as rsgl
from ceciestunepipe.util.spikeextractors.extractors.spikeglxrecordingextractor import spikeglxrecordingextractor as sglex

In [3]:
import spikeinterface as si
import spikeinterface.extractors as se
import spikeinterface.toolkit as st
import spikeinterface.sorters as ss
import spikeinterface.comparison as sc
import spikeinterface.widgets as sw
logger.info('all modules loaded')

2023-03-07 11:28:57,351 root         INFO     all modules loaded


## Session parameters and raw files

#### list all the sessions for this bird

In [4]:
bird = 's_b1515_23'
all_bird_sess = et.list_sessions(bird)
#logger.info('all sessions for bird are {}'.format(all_bird_sess))

### set up bird and sessions parameters
this will define:
- locations of files (for the bird)
- signals and channels to look for in the metadata of the files and in the rig.json parameter file: Note that this have to exist in all of the sessions that will be processed
- 'sess' is unimportant here, but it comes handy if there is need to debug usin a single session

In [6]:
reload(et)
# for one example session

sess_par = {'bird': bird,
           'sess': '2023-03-02',
           'probes': [], #['probe_0'], #probes of interest
           'mic_list': ['microphone_0', 'microphone_1'], #list of mics of interest, by signal name in rig.json
           'adc_list': ['pressure'],
           'stim_list': [], #['wav_syn', 'wav_stim'], # list of adc chans with the stimulus
           'nidq_ttl_list': [], #['wav_ttl'], # list of TTL signals form the nidq digital inputs to extract (besides the 'sync')
           'sort': 0, #label for this sort instance
            'skip_wav': True # skip wavefiles and save streams as npy + dict (for wu.save_wav and long recordings not to blow up)
           }

exp_struct = et.get_exp_struct(sess_par['bird'], sess_par['sess'], sort=sess_par['sort'])

ksort_folder = exp_struct['folders']['ksort']
raw_folder = exp_struct['folders']['sglx']

list all the epochs in a session, to check that it is finding what it has to find

In [7]:
sess_epochs = et.list_sgl_epochs(sess_par)
sess_epochs

2023-03-07 11:29:01,312 ceciestunepipe.file.bcistructure INFO     {'folders': {'bird': '/mnt/sphere/speech_bci/raw_data/s_b1515_23', 'raw': '/mnt/sphere/speech_bci/raw_data/s_b1515_23/2023-03-02', 'sglx': '/mnt/sphere/speech_bci/raw_data/s_b1515_23/2023-03-02/sglx', 'kwik': '/scratch/earneodo/s_b1515_23/sglx/kwik/2023-03-02', 'processed': '/mnt/sphere/speech_bci/processed_data/s_b1515_23/2023-03-02/sglx', 'derived': '/mnt/sphere/speech_bci/derived_data/s_b1515_23/2023-03-02/sglx', 'tmp': '/scratch/earneodo/tmp', 'msort': '/scratch/earneodo/s_b1515_23/sglx/msort/2023-03-02', 'ksort': '/scratch/earneodo/s_b1515_23/sglx/ksort/2023-03-02/0', 'sort': '/mnt/sphere/speech_bci/derived_data/s_b1515_23/2023-03-02/sglx/0'}, 'files': {'par': '/scratch/earneodo/s_b1515_23/sglx/ksort/2023-03-02/0/params.json', 'set': '/mnt/sphere/speech_bci/raw_data/s_b1515_23/2023-03-02/sglx/settings.isf', 'rig': '/mnt/sphere/speech_bci/raw_data/s_b1515_23/2023-03-02/sglx/rig.json', 'kwd': '/scratch/earneodo/s_b151

['0831_g0', '1708_g0']

#### define pre-processing steps for each epoch and for the session

In [20]:
reload(wu)
reload(pre)
one_epoch_dict = pre.preprocess_run(sess_par, exp_struct, sess_epochs[0], skip_wav=sess_par['skip_wav'])

2023-03-06 08:51:59,606 ceciestunepipe.util.spikeextractors.preprocess INFO     PREPROCESSING sess 2023-03-02 | epoch 0831_g0
2023-03-06 08:51:59,607 ceciestunepipe.util.spikeextractors.preprocess INFO     getting extractors
2023-03-06 08:51:59,612 ceciestunepipe.util.spikeextractors.preprocess INFO     Got sglx recordings for keys ['nidq']
2023-03-06 08:51:59,613 ceciestunepipe.util.spikeextractors.preprocess INFO     Getting microphone channel(s) ['microphone_0', 'microphone_1']
2023-03-06 08:52:04,948 ceciestunepipe.util.wavutil INFO     sampling rate 40000
2023-03-06 08:52:04,950 ceciestunepipe.util.wavutil INFO     saving (2, 1241739564)-shaped array as wav in /mnt/sphere/speech_bci/derived_data/s_b1515_23/2023-03-02/sglx/0831_g0/wav_mic.wav
2023-03-06 08:52:04,954 ceciestunepipe.util.wavutil INFO     Not saving the file as wav, going straight to numpy + dict
2023-03-06 08:52:04,955 ceciestunepipe.util.wavutil INFO     saving (2, 1241739564)-shaped array as npy in /mnt/sphere/spee

In [22]:
!ls /mnt/sphere/speech_bci/derived_data/s_b1515_23/2023-03-02/sglx/0831_g0/

nidq_sync_dict.pkl	 wav_stim.npy
nidq_sync_evt.npy	 wav_stimnpy_meta.json
nidq_t0.npy		 wav_stimnpy_meta.pickle
preprocess_par.pickle	 wav_stim_sync_sine_ttl_evt.npy
wav_adc.npy		 wav_stim_sync_sine_ttl.npy
wav_adc-npy_meta.json	 wav_stim_sync_sine_ttl_t.npy
wav_adcnpy_meta.json	 wav_sync_dict.pkl
wav_adc-npy_meta.pickle  wav_sync_evt.npy
wav_adcnpy_meta.pickle	 wav_sync.npy
wav_mic.npy		 wav_sync-npy_meta.json
wav_mic-npy_meta.json	 wav_syncnpy_meta.json
wav_micnpy_meta.json	 wav_sync-npy_meta.pickle
wav_mic-npy_meta.pickle  wav_syncnpy_meta.pickle
wav_micnpy_meta.pickle	 wav_t0.npy
wav_mic.wav


In [23]:
wav_path = '/mnt/sphere/speech_bci/derived_data/s_b1515_23/2023-03-02/sglx/0831_g0/wav_mic.wav'
npy_path = '/mnt/sphere/speech_bci/derived_data/s_b1515_23/2023-03-02/sglx/0831_g0/wav_mic.npy'
pkl_path = '/mnt/sphere/speech_bci/derived_data/s_b1515_23/2023-03-02/sglx/0831_g0/wav_mic-npy_meta.pickle'

In [24]:
pdb

Automatic pdb calling has been turned OFF


In [25]:
wu.read_wav_chan(wav_path, skip_wav=True)

2023-03-06 08:57:58,210 ceciestunepipe.util.wavutil INFO     skipping wav, loading npy instead from /mnt/sphere/speech_bci/derived_data/s_b1515_23/2023-03-02/sglx/0831_g0/wav_mic.npy
2023-03-06 08:57:58,212 ceciestunepipe.util.wavutil INFO     loading npy array from /mnt/sphere/speech_bci/derived_data/s_b1515_23/2023-03-02/sglx/0831_g0/wav_mic.npy


(40000, array([-63,  94,   5, ..., -14,  95, -38], dtype=int16))

In [27]:
### sequentially process all runs of the sessions
def preprocess_session(sess_par: dict):
    logger.info('pre-process all runs of sess ' + sess_par['sess'])
    # get exp struct
    sess_struct = et.get_exp_struct(sess_par['bird'], sess_par['sess'], sort=sess_par['sort'])
    # list the epochs
    sess_epochs = et.list_sgl_epochs(sess_par)
    logger.info('found epochs: {}'.format(sess_epochs))
    # preprocess all epochs
    epoch_dict_list = []
    for i_ep, epoch in enumerate(sess_epochs):
        try:
            exp_struct = et.sgl_struct(sess_par, epoch)
            one_epoch_dict = pre.preprocess_run(sess_par, exp_struct, epoch, skip_wav=sess_par['skip_wav'])
            epoch_dict_list.append(one_epoch_dict)
        except Exception as exc:
            warnings.warn('Error in epoch {}'.format(epoch), UserWarning)
            logger.info(traceback.format_exc)
            logger.info(exc)
            logger.info('Session {} epoch {} could not be preprocessed'.format(sess_par['sess'], epoch))
        
    return epoch_dict_list

all_epoch_list = preprocess_session(sess_par)

2023-03-06 09:00:26,401 root         INFO     pre-process all runs of sess 2023-03-02
2023-03-06 09:00:26,403 ceciestunepipe.file.bcistructure INFO     {'folders': {'bird': '/mnt/sphere/speech_bci/raw_data/s_b1515_23', 'raw': '/mnt/sphere/speech_bci/raw_data/s_b1515_23/2023-03-02', 'sglx': '/mnt/sphere/speech_bci/raw_data/s_b1515_23/2023-03-02/sglx', 'kwik': '/scratch/earneodo/s_b1515_23/sglx/kwik/2023-03-02', 'processed': '/mnt/sphere/speech_bci/processed_data/s_b1515_23/2023-03-02/sglx', 'derived': '/mnt/sphere/speech_bci/derived_data/s_b1515_23/2023-03-02/sglx', 'tmp': '/scratch/earneodo/tmp', 'msort': '/scratch/earneodo/s_b1515_23/sglx/msort/2023-03-02', 'ksort': '/scratch/earneodo/s_b1515_23/sglx/ksort/2023-03-02/0', 'sort': '/mnt/sphere/speech_bci/derived_data/s_b1515_23/2023-03-02/sglx/0'}, 'files': {'par': '/scratch/earneodo/s_b1515_23/sglx/ksort/2023-03-02/0/params.json', 'set': '/mnt/sphere/speech_bci/raw_data/s_b1515_23/2023-03-02/sglx/settings.isf', 'rig': '/mnt/sphere/spee

## Process multiple sessions

In [8]:
sess_list = all_bird_sess
# fist implant, right hemisphere
#sess_list = ['2021-06-24', '2021-06-25', '2021-06-26', '2021-06-27', '2021-06-28', '2021-06-29', '2021-06-30']
sess_list = ['2023-03-02'] # pressure 
sess_list = ['2023-02-22', '2023-02-23', '2023-02-24', '2023-02-27',
            '2023-03-03', '2023-03-04', '2023-03-05'] # pressure 

In [55]:
all_sess_dict = {}

for one_sess in sess_list[:]:
    sess_par['sess'] = one_sess
    preprocess_session(sess_par)

2023-03-06 10:10:49,329 root         INFO     pre-process all runs of sess 2023-02-22
2023-03-06 10:10:49,331 ceciestunepipe.file.bcistructure INFO     {'folders': {'bird': '/mnt/sphere/speech_bci/raw_data/s_b1515_23', 'raw': '/mnt/sphere/speech_bci/raw_data/s_b1515_23/2023-02-22', 'sglx': '/mnt/sphere/speech_bci/raw_data/s_b1515_23/2023-02-22/sglx', 'kwik': '/scratch/earneodo/s_b1515_23/sglx/kwik/2023-02-22', 'processed': '/mnt/sphere/speech_bci/processed_data/s_b1515_23/2023-02-22/sglx', 'derived': '/mnt/sphere/speech_bci/derived_data/s_b1515_23/2023-02-22/sglx', 'tmp': '/scratch/earneodo/tmp', 'msort': '/scratch/earneodo/s_b1515_23/sglx/msort/2023-02-22', 'ksort': '/scratch/earneodo/s_b1515_23/sglx/ksort/2023-02-22/0', 'sort': '/mnt/sphere/speech_bci/derived_data/s_b1515_23/2023-02-22/sglx/0'}, 'files': {'par': '/scratch/earneodo/s_b1515_23/sglx/ksort/2023-02-22/0/params.json', 'set': '/mnt/sphere/speech_bci/raw_data/s_b1515_23/2023-02-22/sglx/settings.isf', 'rig': '/mnt/sphere/spee

## search bouts for those sessions

In [45]:
reload(wu)
reload(sb)
reload(bs)

<module 'ceciestunepipe.util.sound.boutsearch' from '/mnt/cube/earneodo/repos/ceciestunepipe/ceciestunepipe/util/sound/boutsearch.py'>

In [9]:
hparams = { # default parameters work well for starling
    # spectrogram
    'num_freq':1024, #1024# how many channels to use in a spectrogram #
    'preemphasis':0.97, 
    'frame_shift_ms':5, # step size for fft
    'frame_length_ms':10, #128 # frame length for fft FRAME SAMPLES < NUM_FREQ!!!
    'min_level_db':-55, # minimum threshold db for computing spe 
    'ref_level_db':110, # reference db for computing spec
    'sample_rate':None, # sample rate of your data
    
    # spectrograms
    'mel_filter': False, # should a mel filter be used?
    'num_mels':1024, # how many channels to use in the mel-spectrogram
    'fmin': 500, # low frequency cutoff for mel filter
    'fmax': 12000, # high frequency cutoff for mel filter
    
    # spectrogram inversion
    'max_iters':200,
    'griffin_lim_iters':20,
    'power':1.5,

    # Added for the searching
    'read_wav_fun': bs.read_npy_chan, # function for loading the wav_like_stream (has to returns fs, ndarray)
    'file_order_fun': bs.sess_file_id, # function for extracting the file id within the session
    'min_segment': 30, # Minimum length of supra_threshold to consider a 'syllable' (ms)
    'min_silence': 3000, # Minmum distance between groups of syllables to consider separate bouts (ms)
    'min_bout': 3000, # min bout duration (ms)
    'peak_thresh_rms': 0.55, # threshold (rms) for peak acceptance,
    'thresh_rms': 0.25, # threshold for detection of syllables
    'mean_syl_rms_thresh': 0.3, #threshold for acceptance of mean rms across the syllable (relative to rms of the file)
    'max_bout': 180000, #exclude bouts too long
    'l_p_r_thresh': 100, # threshold for n of len_ms/peaks (typycally about 2-3 syllable spans
    
    'waveform_edges': 1000, #get number of ms before and after the edges of the bout for the waveform sample
    
    'bout_auto_file': 'bout_auto.pickle', # extension for saving the auto found files
    'bout_curated_file': 'bout_checked.pickle', #extension for manually curated files (coming soon)
    }

In [10]:
#pick sessions to do the bout searching
# bout_sessions = sess_list[-1:]
# bout_sessions = ['2022-07-16']
bout_sessions = sess_list

In [11]:
pdb

Automatic pdb calling has been turned ON


In [12]:
bout_sessions[0]

'2023-02-22'

In [14]:
reload(sb)

<module 'ceciestunepipe.pipeline.searchbout' from '/mnt/cube/earneodo/repos/ceciestunepipe/ceciestunepipe/pipeline/searchbout.py'>

In [15]:
#all_sessions = sess_list[-1:]
#all_sessions = ['2021-06-15']
for sess in bout_sessions:
    sess_par['sess'] = sess
    sess_bout_pd = sb.get_all_day_bouts(sess_par, hparams, n_jobs=1, ephys_software='sglx', file_ext='npy')
    sb.save_auto_bouts(sess_bout_pd, sess_par, hparams, software='sglx')
    #sess_bouts_folder = os.path.join(exp_struct['folders']['derived'], 'bouts')
    #bouts_to_wavs(sess_bout_pd, sess_par, hparams, sess_bouts_folder)

2023-03-07 11:36:53,107 ceciestunepipe.pipeline.searchbout INFO     Will search for bouts through all session s_b1515_23, 2023-02-22
2023-03-07 11:36:53,563 ceciestunepipe.pipeline.searchbout INFO     getting npy files from /mnt/sphere/speech_bci/derived_data/s_b1515_23/2023-02-22/sglx
2023-03-07 11:36:53,564 ceciestunepipe.pipeline.searchbout INFO     Found 1 files
2023-03-07 11:36:53,566 ceciestunepipe.util.sound.boutsearch INFO     Getting bouts for long file /mnt/sphere/speech_bci/derived_data/s_b1515_23/2023-02-22/sglx/1429_g0/wav_mic.npy
2023-03-07 11:36:53,567 ceciestunepipe.util.wavutil INFO     skipping wav, loading npy instead from /mnt/sphere/speech_bci/derived_data/s_b1515_23/2023-02-22/sglx/1429_g0/wav_mic.npy
2023-03-07 11:36:53,568 ceciestunepipe.util.wavutil INFO     loading npy array from /mnt/sphere/speech_bci/derived_data/s_b1515_23/2023-02-22/sglx/1429_g0/wav_mic.npy
2023-03-07 11:37:00,504 ceciestunepipe.util.sound.boutsearch INFO     splitting file into 7 chunks


  0%|          | 0/7 [00:00<?, ?it/s]

2023-03-07 11:40:29,135 ceciestunepipe.pipeline.searchbout INFO     could not get rate from file /mnt/sphere/speech_bci/derived_data/s_b1515_23/2023-02-22/sglx/1429_g0/wav_mic.npy
2023-03-07 11:40:29,138 ceciestunepipe.pipeline.searchbout INFO     getting spectrograms
2023-03-07 11:41:39,198 ceciestunepipe.pipeline.searchbout INFO     saving bouts pandas to /mnt/sphere/speech_bci/derived_data/s_b1515_23/2023-02-22/bouts_sglx/bout_auto.pickle
2023-03-07 11:41:48,122 ceciestunepipe.pipeline.searchbout INFO     saving bout detect parameters dict to /mnt/sphere/speech_bci/derived_data/s_b1515_23/2023-02-22/bouts_sglx/bout_search_params.pickle
2023-03-07 11:41:48,123 ceciestunepipe.util.sound.boutsearch INFO     saving bout detect parameters dict to /mnt/sphere/speech_bci/derived_data/s_b1515_23/2023-02-22/bouts_sglx/bout_search_params.pickle
2023-03-07 11:41:48,133 ceciestunepipe.pipeline.searchbout INFO     saving bouts pandas to /mnt/sphere/speech_bci/derived_data/s_b1515_23/2023-02-22/b

  0%|          | 0/1 [00:00<?, ?it/s]

2023-03-07 11:41:57,839 ceciestunepipe.util.sound.boutsearch INFO     Getting bouts for long file /mnt/sphere/speech_bci/derived_data/s_b1515_23/2023-02-23/sglx/1432_g0/wav_mic.npy
2023-03-07 11:41:57,841 ceciestunepipe.util.wavutil INFO     skipping wav, loading npy instead from /mnt/sphere/speech_bci/derived_data/s_b1515_23/2023-02-23/sglx/1432_g0/wav_mic.npy
2023-03-07 11:41:57,841 ceciestunepipe.util.wavutil INFO     loading npy array from /mnt/sphere/speech_bci/derived_data/s_b1515_23/2023-02-23/sglx/1432_g0/wav_mic.npy
2023-03-07 11:42:21,365 ceciestunepipe.util.sound.boutsearch INFO     splitting file into 16 chunks


...

2023-03-07 13:08:50,707 ceciestunepipe.pipeline.searchbout INFO     could not get rate from file /mnt/sphere/speech_bci/derived_data/s_b1515_23/2023-03-04/sglx/0839_g0/wav_mic.npy
2023-03-07 13:08:50,924 ceciestunepipe.pipeline.searchbout INFO     could not get rate from file /mnt/sphere/speech_bci/derived_data/s_b1515_23/2023-03-04/sglx/1431_g0/wav_mic.npy
2023-03-07 13:08:50,925 ceciestunepipe.pipeline.searchbout INFO     could not get rate from file /mnt/sphere/speech_bci/derived_data/s_b1515_23/2023-03-04/sglx/2123_g0/wav_mic.npy
2023-03-07 13:08:50,932 ceciestunepipe.pipeline.searchbout INFO     getting spectrograms
2023-03-07 13:11:48,474 ceciestunepipe.pipeline.searchbout INFO     saving bouts pandas to /mnt/sphere/speech_bci/derived_data/s_b1515_23/2023-03-04/bouts_sglx/bout_auto.pickle
2023-03-07 13:12:11,468 ceciestunepipe.pipeline.searchbout INFO     saving bout detect parameters dict to /mnt/sphere/speech_bci/derived_data/s_b1515_23/2023-03-04/bouts_sglx/bout_search_params.

  0%|          | 0/11 [00:00<?, ?it/s]

2023-03-07 13:18:18,291 ceciestunepipe.util.sound.boutsearch INFO     Getting bouts for long file /mnt/sphere/speech_bci/derived_data/s_b1515_23/2023-03-05/sglx/1538_g0/wav_mic.npy
2023-03-07 13:18:18,293 ceciestunepipe.util.wavutil INFO     skipping wav, loading npy instead from /mnt/sphere/speech_bci/derived_data/s_b1515_23/2023-03-05/sglx/1538_g0/wav_mic.npy
2023-03-07 13:18:18,294 ceciestunepipe.util.wavutil INFO     loading npy array from /mnt/sphere/speech_bci/derived_data/s_b1515_23/2023-03-05/sglx/1538_g0/wav_mic.npy
2023-03-07 13:18:31,030 ceciestunepipe.util.sound.boutsearch INFO     splitting file into 14 chunks


  0%|          | 0/14 [00:00<?, ?it/s]

2023-03-07 13:26:14,281 ceciestunepipe.pipeline.searchbout INFO     could not get rate from file /mnt/sphere/speech_bci/derived_data/s_b1515_23/2023-03-05/sglx/0918_g0/wav_mic.npy
2023-03-07 13:26:14,282 ceciestunepipe.pipeline.searchbout INFO     could not get rate from file /mnt/sphere/speech_bci/derived_data/s_b1515_23/2023-03-05/sglx/1538_g0/wav_mic.npy
2023-03-07 13:26:14,289 ceciestunepipe.pipeline.searchbout INFO     getting spectrograms
2023-03-07 13:28:43,886 ceciestunepipe.pipeline.searchbout INFO     saving bouts pandas to /mnt/sphere/speech_bci/derived_data/s_b1515_23/2023-03-05/bouts_sglx/bout_auto.pickle
2023-03-07 13:29:03,223 ceciestunepipe.pipeline.searchbout INFO     saving bout detect parameters dict to /mnt/sphere/speech_bci/derived_data/s_b1515_23/2023-03-05/bouts_sglx/bout_search_params.pickle
2023-03-07 13:29:03,225 ceciestunepipe.util.sound.boutsearch INFO     saving bout detect parameters dict to /mnt/sphere/speech_bci/derived_data/s_b1515_23/2023-03-05/bouts_s

In [50]:
sess_bout_pd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1062 entries, 0 to 1061
Data columns (total 20 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   start_ms        1062 non-null   int64  
 1   end_ms          1062 non-null   int64  
 2   start_sample    1062 non-null   int64  
 3   end_sample      1062 non-null   int64  
 4   p_step          1062 non-null   object 
 5   rms_p           1062 non-null   float64
 6   peak_p          1062 non-null   float64
 7   bout_check      1062 non-null   bool   
 8   file            1062 non-null   object 
 9   len_ms          1062 non-null   int64  
 10  syl_in          1062 non-null   object 
 11  n_syl           1062 non-null   int64  
 12  peaks_p         1062 non-null   object 
 13  n_peaks         1062 non-null   int64  
 14  l_p_ratio       1062 non-null   float64
 15  waveform        1062 non-null   object 
 16  confusing       1062 non-null   bool   
 17  valid_waveform  1062 non-null   b

In [15]:
np.unique(sess_bout_pd['start_ms']).size

883

# debug

## debug search_bout

In [21]:
## look for a single file
sess = sess_list[0]

exp_struct = et.get_exp_struct(sess_par['bird'], sess, ephys_software='sglx')
source_folder = exp_struct['folders']['derived']
wav_path_list = et.get_sgl_files_epochs(source_folder, file_filter='*wav_mic.wav')
wav_path_list.sort()
logger.info('Found {} files'.format(len(wav_path_list)))
print(wav_path_list)

2021-09-22 15:13:39,371 root         INFO     Found 4 files


['/mnt/sphere/speech_bci/derived_data/s_b1253_21/2021-06-14/sglx/0712_g0/wav_mic.wav', '/mnt/sphere/speech_bci/derived_data/s_b1253_21/2021-06-14/sglx/1255_g0/wav_mic.wav', '/mnt/sphere/speech_bci/derived_data/s_b1253_21/2021-06-14/sglx/1740_g0/wav_mic.wav', '/mnt/sphere/speech_bci/derived_data/s_b1253_21/2021-06-14/sglx/2118_g0/wav_mic.wav']


In [22]:
one_file = wav_path_list[0]

In [ ]:
reload(bs)
epoch_bout_pd, epoch_wav = bs.get_bouts_in_long_file(wav_path_list[0], hparams)

2021-09-22 15:13:45,924 ceciestunepipe.util.sound.boutsearch INFO     Getting bouts for long file /mnt/sphere/speech_bci/derived_data/s_b1253_21/2021-06-14/sglx/0712_g0/wav_mic.wav


tu vieja file /mnt/sphere/speech_bci/derived_data/s_b1253_21/2021-06-14/sglx/0712_g0/wav_mic.wav


2021-09-22 15:13:45,962 ceciestunepipe.util.sound.boutsearch INFO     splitting file into 5 chunks


  0%|          | 0/5 [00:00<?, ?it/s]